In [1]:
import pandas as pd
import numpy as np
import importlib

# graphs
import chart_studio.plotly as py
import plotly.graph_objs as go

import util.Plotly as plots

In [2]:
importlib.reload(plots)

<module 'util.Plotly' from 'D:\\scul\\uaic-ub\\licenta\\code\\repr\\util\\Plotly.py'>

# Data

In [3]:
csv_filename= r'data/final_coordinates.csv'

In [4]:
data_df = pd.read_csv(csv_filename)

In [5]:
## convert Broadman Area column values to string
data_df['Broadman Area'] = data_df['Broadman Area'].astype(str)
## add index column
data_df['index_col'] = data_df.index

In [6]:
data_df.head()

,importance,author,title,table_name,keywords,Name,Left/Right,Broadman Area,t,X(R),Y(A),Z(S),z-score,NV,p value,cluster size,index_col
0,++,Cohen et al,Individual differences in extraversion and dop...,Table 1 Study 2 reward - no reward,fMRI; MNI; extraversion; dopamine; money rewar...,R. amygdala,NaN,nan,6.58,18.0,-7.0,-28.0,NaN,NaN,NaN,NaN,0
1,++,Cohen et al,Individual differences in extraversion and dop...,Table 1 Study 2 reward - no reward,fMRI; MNI; extraversion; dopamine; money rewar...,L. amygdala,NaN,nan,6.47,-17.0,-3.0,-14.0,NaN,NaN,NaN,NaN,1
2,++,Cohen et al,Individual differences in extraversion and dop...,Table 1 Study 2 reward - no reward,fMRI; MNI; extraversion; dopamine; money rewar...,L. hippocampus,NaN,nan,5.40,-28.0,-24.0,-14.0,NaN,NaN,NaN,NaN,2
3,++,Cohen et al,Individual differences in extraversion and dop...,Table 1 Study 2 reward - no reward,fMRI; MNI; extraversion; dopamine; money rewar...,L. orbitofrontal cortex,NaN,11,5.31,-21.0,21.0,-25.0,NaN,NaN,NaN,NaN,3
4,++,Cohen et al,Individual differences in extraversion and dop...,Table 1 Study 2 reward - no reward,fMRI; MNI; extraversion; dopamine; money rewar...,R. precuneus,NaN,23,5.21,4.0,-56.0,35.0,NaN,NaN,NaN,NaN,4


In [7]:
data_df.shape

(656, 17)

In [8]:
data_df.columns.values.tolist()

['importance',
 'author',
 'title',
 'table_name',
 'keywords',
 'Name',
 'Left/Right',
 'Broadman Area',
 't',
 'X(R)',
 'Y(A)',
 'Z(S)',
 'z-score',
 'NV',
 'p value',
 'cluster size',
 'index_col']

In [9]:
coordinates = data_df[['X(R)', 'Y(A)', 'Z(S)']]


# Separate BA of interest

In [84]:
# df_ba = data_df[data_df['Broadman Area'].str.contains('8|9|10|11|12|13|14|24|25|32|33|44|45|46|47')]
# df_ba = data_df[data_df['Broadman Area'].str.contains('4|6|8|9|10|11|12|25')]
df_ba = data_df[data_df['Broadman Area'].str.contains('^4$|^6$|^8$|^9$|^10$|^11$|^12$|^25$',regex=True)]
df_ba_nan = data_df[data_df['Broadman Area'].str.contains('^nan$',regex=True)]
assert(len(df_ba_nan['Broadman Area'].unique())==1)

In [85]:
df = df_ba

col1='X(R)'
col2='Y(A)'
col3='Z(S)'
Xn = df[col1].to_numpy()  # x-coordinates of nodes
Yn = df[col2].to_numpy()   # y-coordinates
Zn = df[col3].to_numpy()

In [88]:
Xn_nan = df_ba_nan[col1].to_numpy()
Yn_nan = df_ba_nan[col2].to_numpy()
Zn_nan = df_ba_nan[col3].to_numpy()

In [86]:
# color V1
ba = df['Broadman Area'].unique()
ba_idx = {str(x): ba.tolist().index(x) for x in ba}
no_colors = len(ba)
color_group_id = []
labels = []
for i, val in enumerate(df[['Broadman Area','index_col','Name']].to_numpy()):
    color_group_id.append(ba_idx[str(val[0])])
    labels.append('index: {index}; ba: {ba}; name: {name}'.format(index=str(val[1]),name=val[2], ba=str(val[0])))

In [100]:
# color V2
## Alphabet maximum number of colors = 26
ba = df['Broadman Area'].unique()
ba_idx = {str(x): ba.tolist().index(x) for x in ba}
no_colors = len(ba)
color_group = []
labels = []
labels_nan = []

#colors
import plotly.express as px
color_list = px.colors.qualitative.Alphabet[:no_colors]
for i, val in enumerate(df[['Broadman Area','index_col','Name']].to_numpy()):
    # print(i,val[0])
    if val[0] == 'nan':
        color_group.append('#ebeded')
    else:
        color_group.append(color_list[ba_idx[str(val[0])]] )
    labels.append('index: {index}; ba: {ba}; name: {name}'.format(index=str(val[1]),name=val[2], ba=str(val[0])))

#labels for nan
for i,val in enumerate(df_ba_nan[['Broadman Area','index_col','Name']].to_numpy()):
        labels_nan.append('index: {index}; ba: {ba}; name: {name}'.format(index=str(val[1]),name=val[2], ba=str(val[0])))

In [101]:
color_nan = '#ebeded'

trace1 = go.Scatter3d(x=Xn,
                      y=Yn,
                      z=Zn,
                      mode='markers',
                      name='actors',
                      marker=dict(symbol='circle',
                                  size=6,
                                  color = color_group,
                                  # color=color_group_id,
                                  # colorscale='Viridis',
                                  line=dict(color='rgb(50,50,50)', width=2)
                                  ),
                      text=labels,
                      hoverinfo='text'
                      )

#plot nans separately
trace2 = go.Scatter3d(x=Xn_nan,
                      y=Yn_nan,
                      z=Zn_nan,
                      mode='markers',
                      name='actors',
                      opacity=0.5,
                      marker=dict(symbol='circle',
                                  size=6,
                                  color = color_nan,
                                  line=dict(color='#acadad', width=0.5)
                                  ),
                      text=labels_nan,
                      hoverinfo='text'
                      )
axis = dict(showbackground=False,
            showline=False,
            zeroline=False,
            showgrid=False,
            showticklabels=False,
            title=''
            )

layout = go.Layout(
    title="Center Coordinates",
    width=1500,
    height=1500,
    showlegend=False,
    scene=dict(
        xaxis=dict(axis),
        yaxis=dict(axis),
        zaxis=dict(axis),
    ),
    margin=dict(
        t=100
    ),
    hovermode='closest',
    annotations=[
        dict(
            showarrow=False,
            text="Data source: " +csv_filename,
            xref='paper',
            yref='paper',
            x=0,
            y=0.1,
            xanchor='left',
            yanchor='bottom',
            font=dict(
                size=14
            )
        )
    ], )

In [72]:
## Open in Browser
import plotly.io as pio
pio.renderers.default = "browser"

In [102]:
data = [trace1,trace2]
fig = go.Figure(data=data, layout=layout,layout_title_text='Center Coordinates')
# py.iplot(fig, filename='Les-Miserables')
fig.show()